In [1]:
%pip install tensorflow


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

data = pd.read_csv('data.csv')



label_enc = LabelEncoder()
data['Player Role'] = label_enc.fit_transform(data['Player Role'])
data['Team'] = label_enc.fit_transform(data['Team'])

features = ['Runs Scored', 'Wickets Taken', 'Balls Faced', 'Balls Bowled', 
            'Overs Bowled', 'Maidens Bowled', 'Runs Conceded', 
            'Batting Average', 'Bowling Average', 'Strike Rate', 'Economy Rate', 
            'Centuries Scored', 'Half Centuries Scored', 'Ducks Scored', 
            'Wickets Taken Last Match', 'Runs Scored Last Match', 
            'Player Age', 'Player Experience', 'Player Role', 'Team']
X = data[features]
y = data['In Playing 11']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(X_train.shape[1], 1), activation='relu'))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

X_train_lstm = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_lstm = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, validation_data=(X_test_lstm, y_test))


Epoch 1/50
1/1 [==============================] - 3s 3s/step - loss: 0.7229 - accuracy: 0.2609 - val_loss: 0.7123 - val_accuracy: 0.3333
Epoch 2/50
1/1 [==============================] - 0s 58ms/step - loss: 0.7155 - accuracy: 0.2609 - val_loss: 0.7081 - val_accuracy: 0.3333
Epoch 3/50
1/1 [==============================] - 0s 82ms/step - loss: 0.7084 - accuracy: 0.2609 - val_loss: 0.7041 - val_accuracy: 0.1667
Epoch 4/50
1/1 [==============================] - 0s 62ms/step - loss: 0.7017 - accuracy: 0.2174 - val_loss: 0.7002 - val_accuracy: 0.1667
Epoch 5/50
1/1 [==============================] - 0s 79ms/step - loss: 0.6951 - accuracy: 0.3478 - val_loss: 0.6964 - val_accuracy: 0.3333
Epoch 6/50
1/1 [==============================] - 0s 75ms/step - loss: 0.6888 - accuracy: 0.7391 - val_loss: 0.6927 - val_accuracy: 0.8333
Epoch 7/50
1/1 [==============================] - 0s 79ms/step - loss: 0.6827 - accuracy: 0.7826 - val_loss: 0.6891 - val_accuracy: 0.6667
Epoch 8/50
1/1 [=============

In [12]:
from tensorflow.keras.layers import GRU

gru_model = Sequential()
gru_model.add(GRU(64, input_shape=(X_train.shape[1], 1), activation='relu'))
gru_model.add(Dense(1, activation='sigmoid'))

gru_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

X_train_gru = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_gru = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

gru_model.fit(X_train_gru, y_train, epochs=50, batch_size=32, validation_data=(X_test_gru, y_test))


Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 0.7164 - accuracy: 0.2609 - val_loss: 0.7104 - val_accuracy: 0.3333
Epoch 2/50
1/1 [==============================] - 0s 60ms/step - loss: 0.7102 - accuracy: 0.2609 - val_loss: 0.7065 - val_accuracy: 0.3333
Epoch 3/50
1/1 [==============================] - 0s 61ms/step - loss: 0.7042 - accuracy: 0.2174 - val_loss: 0.7027 - val_accuracy: 0.1667
Epoch 4/50
1/1 [==============================] - 0s 62ms/step - loss: 0.6982 - accuracy: 0.3043 - val_loss: 0.6990 - val_accuracy: 0.1667
Epoch 5/50
1/1 [==============================] - 0s 107ms/step - loss: 0.6924 - accuracy: 0.5217 - val_loss: 0.6954 - val_accuracy: 0.6667
Epoch 6/50
1/1 [==============================] - 0s 74ms/step - loss: 0.6867 - accuracy: 0.7826 - val_loss: 0.6919 - val_accuracy: 0.6667
Epoch 7/50
1/1 [==============================] - 0s 69ms/step - loss: 0.6811 - accuracy: 0.6957 - val_loss: 0.6885 - val_accuracy: 0.6667
Epoch 8/50
1/1 [============

In [13]:
import numpy as np

X_all = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

lstm_predictions = lstm_model.predict(X_all).flatten()
gru_predictions = gru_model.predict(X_all).flatten()
print(lstm_predictions)
print(gru_predictions)
combined_predictions = (lstm_predictions + gru_predictions) / 2

data['Combined Predictions'] = combined_predictions

top_players = data.sort_values(by='Combined Predictions', ascending=False)

selected_players = []
selected_teams = set()

for team in data['Team'].unique():
    team_player = top_players[top_players['Team'] == team].iloc[0]
    selected_players.append(team_player)
    selected_teams.add(team)

remaining_spots = 11 - len(selected_players)
for index, player in top_players.iterrows():
    if len(selected_players) < 11 and player['Player Name'] not in [p['Player Name'] for p in selected_players]:
        selected_players.append(player)

selected_players_df = pd.DataFrame(selected_players)

print(selected_players_df[['Player Name', 'Team', 'Combined Predictions']])


1/1 [==============================] - 0s 317ms/step
[1.         1.         0.7244586  0.72959596 0.73285    0.9238206
 0.99999887 0.73230475 0.8535609  0.6449536  0.62858593 0.738696
 0.6798373  0.8798884  1.         0.6125645  0.6500313  0.6296426
 0.98298    0.6584312  0.71263635 0.64512587 0.8437567  0.62486935
 1.         0.6303844  0.6873883  1.         0.9866459 ]
[0.99820143 0.9802072  0.7253051  0.74015784 0.71163017 0.73327
 0.91827255 0.7275993  0.7429654  0.7150576  0.7135253  0.744188
 0.71818227 0.753551   0.9712682  0.70648086 0.6949824  0.7090286
 0.7903555  0.723845   0.6938796  0.70221066 0.7543398  0.70709676
 0.75693995 0.6950789  0.7463671  0.99787974 0.7334981 ]
           Player Name  Team  Combined Predictions
0          Virat Kohli     0              0.999101
14  Maheesh Theekshana     1              0.985634
27       Kuldeep Yadav     0              0.998940
1         Rohit Sharma     0              0.990104
6    Washington Sundar     0              0.959136
1

C:\Users\shoya\AppData\Local\Temp\ipykernel_19876\1278156176.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[col] = y[col] / data['Player Experience']


Epoch 1/50
1/1 [==============================] - 14s 14s/step - loss: 0.9970 - mae: 0.7267 - val_loss: 1.0115 - val_mae: 0.7447
Epoch 2/50
1/1 [==============================] - 0s 80ms/step - loss: 0.9910 - mae: 0.7235 - val_loss: 1.0143 - val_mae: 0.7460
Epoch 3/50
1/1 [==============================] - 0s 76ms/step - loss: 0.9858 - mae: 0.7207 - val_loss: 1.0162 - val_mae: 0.7470
Epoch 4/50
1/1 [==============================] - 0s 76ms/step - loss: 0.9810 - mae: 0.7184 - val_loss: 1.0173 - val_mae: 0.7479
Epoch 5/50
1/1 [==============================] - 0s 74ms/step - loss: 0.9764 - mae: 0.7156 - val_loss: 1.0181 - val_mae: 0.7490
Epoch 6/50
1/1 [==============================] - 0s 73ms/step - loss: 0.9716 - mae: 0.7123 - val_loss: 1.0187 - val_mae: 0.7503
Epoch 7/50
1/1 [==============================] - 0s 99ms/step - loss: 0.9662 - mae: 0.7084 - val_loss: 1.0187 - val_mae: 0.7515
Epoch 8/50
1/1 [==============================] - 0s 81ms/step - loss: 0.9605 - mae: 0.7040 - val

Top 11 Players based on Predicted Fantasy Points (with role validation):
           Player Name  Team  Predicted Fantasy Points  Player Role
20        Shreyas Iyer     0                137.000000            2
16        Sanju Samson     0                107.000000            2
18     Janith Liyanage     1                 90.600000            2
5         Shubman Gill     0                 78.600000            0
6    Washington Sundar     0                 77.899999            3
22         Shivam Dube     0                 77.500000            2
21         Rinku Singh     0                 58.000000            2
12     Pathum Nissanka     1                 53.100000            0
13        Kusal Mendis     1                 48.200000            2
4     Suryakumar Yadav     0                 45.200000            0
0          Virat Kohli     0                 45.100000            0
14  Maheesh Theekshana     1                 34.500000            1

Predicted Stats for Top Players (per match